In [3]:
import nnsight

In [36]:
model = nnsight.LanguageModel("gpt2-large", device_map="cuda:0")
tokenizer = model.tokenizer

config.json: 100%|██████████| 666/666 [00:00<00:00, 145kB/s]
vocab.json: 100%|██████████| 1.04M/1.04M [00:00<00:00, 17.3MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 51.9MB/s]
tokenizer.json: 100%|██████████| 1.36M/1.36M [00:00<00:00, 34.9MB/s]


In [37]:
from lens.lens import TunedLens
lens = TunedLens(model)
lens.load_states()

params.pt: 100%|██████████| 236M/236M [00:05<00:00, 39.9MB/s] 
lens/gpt2-large/config.json: 100%|██████████| 198/198 [00:00<00:00, 76.2kB/s]


<All keys matched successfully>


In [38]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-35): 36 x GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1280, out_features=50257, bias=False)
)

In [39]:
def decoder(x):
    # decoder consists of final layer norm + unembedding
    return model.lm_head(model.transformer.ln_f(x))
    # return model.lm_head(x)

saved = []

with model.invoke("it was the best of times, it was the worst of times") as invoker:
    for layer in range(len(model.transformer.h)):
        out = model.transformer.h[layer].output[0]
        saved.append(decoder(lens(out, layer)).save())

model.safetensors: 100%|██████████| 3.25G/3.25G [00:42<00:00, 77.0MB/s]
generation_config.json: 100%|██████████| 124/124 [00:00<00:00, 26.1kB/s]
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [40]:
saved = [s.value.detach().cpu() for s in saved]

In [41]:
new = []
for s in saved:
    temp = []
    s = s.softmax(-1).argmax(-1)
    for c in s:
        temp.append(tokenizer.decode(c))

    new.append(temp)
    

In [42]:
new

[[', a entry best opportunity times the was not moment worst expectation times'],
 ['\xa0 a the to the, and. a., the,'],
 [', a\n of the, and was a\n, the,'],
 ['ous a\n of the, and was a world, the,'],
 ['ine a\n way the, and was a right thing the,'],
 ['ish a first of the, and was a first of the,'],
 ['- a first way the, and was a first of the.'],
 ['- a first way the, but was a first of the.'],
 ['- a first way the, but was a first thing the.'],
 ['- a first way the, but was a first thing the.'],
 ['� not first way the, but was a first thing the.'],
 ["� a first way the, but's a first of the."],
 ["� not first way the, but's a perfect of the,"],
 ["� not first way the, but's a perfect of the,"],
 ["� not first way the, but's a perfect of the,"],
 ["� not first way the, but's a perfect of the,"],
 ["� a first way the and and's a perfect of luck,"],
 ["� a first way the, and's a perfect of luck,"],
 ["� a first way the, and's a perfect of luck,"],
 ["rd a first thing the, and's the wo